In [8]:
# install packages
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install mljar-supervised

  Using cached lightgbm-3.0.0-py2.py3-none-win_amd64.whl (737 kB)
  Using cached xgboost-1.3.3-py3-none-win_amd64.whl (95.2 MB)
  Using cached scipy-1.6.1-cp37-cp37m-win_amd64.whl (32.6 MB)
  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)
  Using cached cloudpickle-1.3.0-py2.py3-none-any.whl (26 kB)
  Using cached numpy-1.20.3-cp37-cp37m-win_amd64.whl (13.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


ERROR: Could not install packages due to an OSError: [WinError 5] 存取被拒。: 'C:\\Users\\Chen\\AppData\\Local\\Temp\\pip-uninstall-ftzi5woy\\libopenblas.wcdjnk7yvmpzq2me2zzhjjrj3jikndb7.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [2]:
# import packages
import os
import numpy as np
import pandas as pd

# mljar
from supervised.automl import AutoML

import catboost
import lightgbm
import xgboost

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from joblib import load, dump

In [3]:
# # connect with Google Cloud
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/colab/TPS May"
path = r'C:\Users\Chen\Desktop\Kaggle\Regression\Tabular Playground Series - Jan 2021'
os.chdir(path)

In [4]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
all_data = pd.concat([df_train, df_test]).reset_index(drop=True)

In [6]:
# target labelling
y = pd.DataFrame(df_train['target']).astype('float64')

col = [i for i in df_train.columns if i not in ['id', 'target']]
scaler = StandardScaler()
num_df = pd.DataFrame(scaler.fit_transform(all_data[col]), columns=col)

# create new feature by Kmean cluster
clus = KMeans(n_clusters=4, random_state=42, max_iter=1000, n_jobs=-1)
clus.fit(num_df)
num_df['kmeans'] = clus.labels_

df_train = num_df[:len(df_train)]
df_test = num_df[len(df_train):]

x = df_train
x = x.astype('float64')

'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).


In [7]:
# construct model
# AutoML : mljar
RESULTS_PATH = 'mljar-20210526'
SEED = 20210526

cv = {"validation_type": "kfold",
      "k_folds": 5,
      "shuffle": True,
      "stratify": False,
      "random_seed": SEED}

automl = AutoML(results_path=RESULTS_PATH,
                mode="Optuna",                          # or 'Explain', 'Perform', 'Compete'
                ml_task='regression',   # or 'auto', 'binary_classification', 'regression'
                algorithms=['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors'],
                train_ensemble=True,
                stack_models=True,
                eval_metric='rmse',
                validation_strategy=cv,
                golden_features=True,
                boost_on_errors=True,
                optuna_time_budget=60*60,
                total_time_limit=8*60*60,
                optuna_verbose=False,
                n_jobs=-1,
                random_state=SEED)

In [8]:
# training model
automl.fit(x, y)

Linear algorithm was disabled.
AutoML directory: mljar-20210526
Expected computing time:
Total training time: Optuna + ML training = 61200 seconds
Total Optuna time: len(algorithms) * optuna_time_budget = 32400 seconds
Total ML model training time: 28800 seconds
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble availabe models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'golden_features', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 0.733071 trained in 1.77 seconds
2_DecisionTree rmse 0.726533 trained in 8.47 seconds
* Step default_algorithms will try to check up to 6 models
3_Optuna_LightGBM rmse 0.696432 trained in 336.58 seconds
4_Optuna_Xgboost rmse 0.696238 trained in 255.

AutoML(algorithms=['Baseline', 'Linear', 'Decision Tree', 'Random Forest',
                   'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost',
                   'Neural Network', 'Nearest Neighbors'],
       boost_on_errors=True, eval_metric='rmse', golden_features=True,
       ml_task='regression', mode='Optuna', optuna_time_budget=3600,
       optuna_verbose=False, random_state=20210526,
       results_path='mljar-20210526', stack_models=True, total_time_limit=28800,
       validation_strategy={'k_folds': 5, 'random_seed': 20210526,
                            'shuffle': True, 'stratify': False,
                            'validation_type': 'kfold'})

In [9]:
# load well-trained model
automl = AutoML(results_path=RESULTS_PATH)

In [10]:
# predict probability
x_test = df_test
x_test = x_test.astype('float64')
result = automl.predict(x_test)

In [11]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result
sub.to_csv(f'{RESULTS_PATH}.csv', index=False)